# 04_05_reduce_light_frame_using_master_files_ys

## 필요한 모듈

이 프로젝트를 위해서는 아래의 모듈이 필요하다. 

> numpy, pandas, matplotlib, astropy, version_information

### 모듈 설치

1. 콘솔 창에서 모듈을 설치할 때는 아래와 같은 형식으로 입력하면 된다.

>pip install module_name==version

>conda install module_name==version

2. 주피터 노트북(코랩 포함)에 설치 할 때는 아래의 셀을 실행해서 실행되지 않은 모듈을 설치할 수 있다. (pip 기준) 만약 아나콘다 환경을 사용한다면 7행을 콘다 설치 명령어에 맞게 수정하면 된다.

In [1]:
# Install a pip package in the current Jupyter kernel
import importlib, sys, subprocess
print(f"sys.executable: {sys.executable}")
#!{sys.executable} -m pip install numpy==1.22 pillow==8.4 matplotlib==3.2

packages = "numpy, pandas, matplotlib, scipy, astropy, photutils, ccdproc, version_information" # required modules
pkgs = packages.split(", ")
for pkg in pkgs :
    if not importlib.util.find_spec(pkg):
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg, '-q'])
        print(f"**** {pkg} module is now installed.")
    else: 
        print(f"******** {pkg} module is already installed.")
%load_ext version_information
import time
now = time.strftime("%Y-%m-%d %H:%M:%S (%Z = GMT%z)")
print(f"This notebook was generated at {now} ")

vv = %version_information {packages}
for i, pkg in enumerate(vv.packages):
    print(f"{i} {pkg[0]:10s} {pkg[1]:s}")


sys.executable: /home/guitar79/anaconda3/envs/astro_Python_env/bin/python
******** numpy module is already installed.
******** pandas module is already installed.
******** matplotlib module is already installed.
******** scipy module is already installed.
******** astropy module is already installed.
******** photutils module is already installed.
******** ccdproc module is already installed.
******** version_information module is already installed.
This notebook was generated at 2024-11-28 10:44:17 (KST = GMT+0900) 
0 Python     3.12.3 64bit [GCC 11.2.0]
1 IPython    8.25.0
2 OS         Linux 6.8.0 49 generic x86_64 with glibc2.39
3 numpy      1.26.4
4 pandas     2.2.2
5 matplotlib 3.8.4
6 scipy      1.13.1
7 astropy    6.1.0
8 photutils  1.12.0
9 ccdproc    2.4.2
10 version_information 1.0.4


### 모듈 버전 확인

아래 셀을 실행하면 이 노트북을 실행한 파이썬 및 관련 모듈의 버전을 확인할 수 있다.

### import modules

In [2]:
#%%
from glob import glob
from pathlib import Path
import os
import shutil
import numpy as np
import astropy.units as u
from astropy.stats import sigma_clip
from ccdproc import combine, ccd_process, CCDData

import ysfitsutilpy as yfu
#import ysphotutilpy as ypu
#import ysvisutilpy as yvu

import _astro_utilities
import _Python_utilities

In [3]:
#%%
#######################################################
BASEDIR = Path("/mnt/Rdata/OBS_data")  

PROJECDIR = BASEDIR / "C1-Variable"
TODODIR = PROJECDIR / "-_-_-_2016-_-_RiLA600_STX-16803_-_2bin"
TODODIR = PROJECDIR / "-_-_-_2017-01_-_RiLA600_STX-16803_-_2bin"
TODODIR = PROJECDIR / "-_-_-_2017-03_-_RiLA600_STX-16803_-_2bin"
TODODIR = PROJECDIR / "-_-_-_2017-05_-_RiLA600_STX-16803_-_2bin"
# TODODIR = PROJECDIR / "-_-_-_2017-06_-_RiLA600_STX-16803_-_2bin"
# TODODIR = PROJECDIR / "-_-_-_2021-10_-_RiLA600_STX-16803_-_2bin"
# TODODIR = PROJECDIR / "-_-_-_2022-01_-_RiLA600_STX-16803_-_2bin"

# PROJECDIR = BASEDIR / "C2-Asteroid"
# TODODIR = PROJECDIR / "-_-_-_2022-_-_GSON300_STF-8300M_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2022-_-_RiLA600_STX-16803_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2022-_-_RiLA600_STX-16803_-_2bin"
# TODODIR = PROJECDIR / "-_-_-_2023-_-_GSON300_STF-8300M_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2023-_-_RiLA600_STX-16803_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2023-_-_RiLA600_STX-16803_-_2bin"

# PROJECDIR = BASEDIR / "C3-EXO"
# TODODIR = PROJECDIR / "-_-_-_2024-05_-_GSON300_STF-8300M_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2024-05_-_RiLA600_STX-16803_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2024-06_-_RiLA600_STX-16803_-_2bin"

# PROJECDIR = BASEDIR / "C4-Spectra"
# TODODIR = PROJECDIR / "-_-_-_2024-05_TEC140_ASI183MMPro_-_1bin"

DOINGDIRs = sorted(_Python_utilities.getFullnameListOfsubDirs(TODODIR))
print ("DOINGDIRs: ", format(DOINGDIRs))
print ("len(DOINGDIRs): ", format(len(DOINGDIRs)))

try : 
    BDFDIR = [x for x in DOINGDIRs if "CAL-BDF" in str(x)]
    print ("BDFDIR: ", format(BDFDIR))
    MASTERDIR = Path(BDFDIR[0]) / _astro_utilities.master_dir
    if not MASTERDIR.exists():
        os.makedirs("{}".format(str(MASTERDIR)))
        print("{} is created...".format(str(MASTERDIR)))
    print ("MASTERDIR: ", format(MASTERDIR))
except : 
    pass

DOINGDIRs = sorted([x for x in DOINGDIRs if "_LIGHT_" in str(x)])
# print ("DOINGDIRs: ", format(DOINGDIRs))
# print ("len(DOINGDIRs): ", format(len(DOINGDIRs)))

# filter_str = 'BL'
# DOINGDIRs = [x for x in DOINGDIRs if filter_str in str(x)]
# remove = 'BIAS'
# DOINGDIRs = [x for x in DOINGDIRs if remove not in x]
# remove = 'DARK'
# DOINGDIRs = [x for x in DOINGDIRs if remove not in x]
# remove = 'FLAT'
# DOINGDIRs = [x for x in DOINGDIRs if remove not in x]
print ("DOINGDIRs: ", DOINGDIRs)
print ("len(DOINGDIRs): ", len(DOINGDIRs))

DOINGDIRs:  ['/mnt/Rdata/OBS_data/C1-Variable/-_-_-_2017-05_-_RiLA600_STX-16803_-_2bin/-_CAL-BDF_-_2017-05_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/OBS_data/C1-Variable/-_-_-_2017-05_-_RiLA600_STX-16803_-_2bin/NSV-7244_LIGHT_-_2017-05-19_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/OBS_data/C1-Variable/-_-_-_2017-05_-_RiLA600_STX-16803_-_2bin/NSV-7244_LIGHT_-_2017-05-21_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/OBS_data/C1-Variable/-_-_-_2017-05_-_RiLA600_STX-16803_-_2bin/NSV-7244_LIGHT_-_2017-05-22_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/OBS_data/C1-Variable/-_-_-_2017-05_-_RiLA600_STX-16803_-_2bin/NSV-7244_LIGHT_-_2017-05-24_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/OBS_data/C1-Variable/-_-_-_2017-05_-_RiLA600_STX-16803_-_2bin/NSV-9237_LIGHT_-_2017-05-18_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/OBS_data/C1-Variable/-_-_-_2017-05_-_RiLA600_STX-16803_-_2bin/NSV-9237_LIGHT_-_2017-05-25_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/OBS_data/C1-Variable/-_-_-_2017-05_-_RiLA600_STX-16803_-_2b

In [ ]:
tryagain = False
trynightsky = True

for DOINGDIR in DOINGDIRs[:1] :
    DOINGDIR = Path(DOINGDIR)
    print(f"Starting: {str(DOINGDIR.parts[-1])}")
    
    sMASTERDIR = DOINGDIR / _astro_utilities.master_dir
    REDUCEDDIR = DOINGDIR / _astro_utilities.reduced_dir
    REDUC_nightsky = DOINGDIR / _astro_utilities.reduced_nightsky_dir

    if not sMASTERDIR.exists():
        os.makedirs(str(sMASTERDIR))
        print("{} is created...".format(str(sMASTERDIR)))

    if not REDUCEDDIR.exists():
        os.makedirs(str(REDUCEDDIR))
        print("{} is created...".format(str(REDUCEDDIR)))

    if not REDUC_nightsky.exists():
        os.makedirs("{}".format(str(REDUC_nightsky)))
        print("{} is created...".format(str(REDUC_nightsky)))

    summary = yfu.make_summary(DOINGDIR/"*.fit*")
    if summary is not None :
        #print(summary)
        print("len(summary):", len(summary))
        print("summary:", summary)
        #print(summary["file"][0])

        df_light = summary.loc[summary["IMAGETYP"] == "LIGHT"].copy()
        df_light = df_light.reset_index(drop=True)

        summary_master = yfu.make_summary(MASTERDIR/"*.fit*", 
                           verbose = False,
                           )
        # print("summary_master", summary_master)

        summary_master_dark = summary_master.loc[summary_master["IMAGETYP"] == "DARK"].copy()
        summary_master_dark.reset_index(inplace=True)
        # print("summary_master_dark", summary_master_dark)

        if 'EXPTIME' in summary_master_dark :
            check_exptimes = summary_master_dark['EXPTIME'].drop_duplicates()
            check_exptimes = check_exptimes.reset_index(drop=True)
        print("check_exptimes", check_exptimes)

        for _, row in df_light.iterrows():

            fpath = Path(row["file"])
            ccd = yfu.load_ccd(fpath)
            filt = ccd.header["FILTER"]
            expt = ccd.header["EXPTIME"]

Starting: NSV-7244_LIGHT_-_2017-05-19_-_RiLA600_STX-16803_-_2bin
All 98 keywords (guessed from /mnt/Rdata/OBS_data/C1-Variable/-_-_-_2017-05_-_RiLA600_STX-16803_-_2bin/NSV-7244_LIGHT_-_2017-05-19_-_RiLA600_STX-16803_-_2bin/NSV-7244_LIGHT_B_2017-05-19-12-05-03_70sec_RiLA600_STX-16803_-20c_2bin.fit) will be loaded.


DELMAG   =   0.0000            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.1773            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.1971            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.1899            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.1942            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.1855            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.1688            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.1805            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.1836            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.1787            /Veriable - Comparison,   Delmag 

len(summary): 675
summary:                                                   file  filesize  SIMPLE  \
0    /mnt/Rdata/OBS_data/C1-Variable/-_-_-_2017-05_...   8432640    True   
1    /mnt/Rdata/OBS_data/C1-Variable/-_-_-_2017-05_...   8432640    True   
2    /mnt/Rdata/OBS_data/C1-Variable/-_-_-_2017-05_...   8432640    True   
3    /mnt/Rdata/OBS_data/C1-Variable/-_-_-_2017-05_...   8432640    True   
4    /mnt/Rdata/OBS_data/C1-Variable/-_-_-_2017-05_...   8432640    True   
..                                                 ...       ...     ...   
670  /mnt/Rdata/OBS_data/C1-Variable/-_-_-_2017-05_...   8432640    True   
671  /mnt/Rdata/OBS_data/C1-Variable/-_-_-_2017-05_...   8432640    True   
672  /mnt/Rdata/OBS_data/C1-Variable/-_-_-_2017-05_...   8432640    True   
673  /mnt/Rdata/OBS_data/C1-Variable/-_-_-_2017-05_...   8432640    True   
674  /mnt/Rdata/OBS_data/C1-Variable/-_-_-_2017-05_...   8432640    True   

     BITPIX  NAXIS  NAXIS1  NAXIS2  BSCALE    BZERO OBSERVAT

DELMAG   =  -0.1773            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.1971            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.1899            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.1942            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.1855            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.1688            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.1805            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.1836            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.1787            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.1823            /Veriable - Comparison,   Delmag 

In [5]:
fpath = Path(row["file"])
ccd = yfu.load_ccd(fpath)
filt = ccd.header["FILTER"]
expt = ccd.header["EXPTIME"]

In [ ]:
not False 
tryagain = True
trynightsky = True 

In [ ]:
idx = abs(summary_master_dark['EXPTIME'] - expt).idxmin()
print(idx)

3


In [15]:
if (REDUCEDDIR / fpath.name).exists() and not tryagain :
    print(f"reduction file already exists...\n{fpath.name}")
    pass
else :
    try : 
        if not (MASTERDIR / f"master_flat_{filt.upper()}_norm.fits").exists() :
            print(f"{MASTERDIR}/master_flat_{filt.upper()}_norm.fits is not exists...")
        else :
            if (MASTERDIR / f"master_dark_{expt:.0f}sec.fits").exists() :
                print(f"Reduce with master_dark_{expt:.0f}sec.fits ...")

                red = yfu.ccdred(
                    ccd,
                    output=Path(f"{REDUCEDDIR/ fpath.name}"),
                    mdarkpath=str(MASTERDIR / "master_dark_{:.0f}sec.fits".format(expt)),
                    mflatpath=str(MASTERDIR / "master_flat_{}_norm.fits".format(filt.upper())),
                    # flat_norm_value=1,  # 1 = skip normalization, None = normalize by mean
                    overwrite=True,
                    )
            else : 
                print(f"Reduce with master_dark_{summary_master_dark['EXPTIME'][idx]:.0f}sec.fits is not exists...")
                red = yfu.ccdred(
                    ccd,
                    output=Path(f"{REDUCEDDIR/ fpath.name}"),
                    mdarkpath=str(MASTERDIR / f"master_dark_{summary_master_dark['EXPTIME'][idx]:.0f}sec.fits"),
                    mflatpath=str(MASTERDIR / f"master_flat_{filt.upper()}_norm.fits"),
                    dark_scale = True,
                    exptime_dark = summary_master_dark['EXPTIME'][idx],
                    # flat_norm_value=1,  # 1 = skip normalization, None = normalize by mean
                    overwrite=True,
                    )

    except Exception as e: 
        print("err:", e)
        pass

err: name 'idx' is not defined


In [14]:
for DOINGDIR in DOINGDIRs[:1] :
    DOINGDIR = Path(DOINGDIR)
    print(f"Starting: {str(DOINGDIR.parts[-1])}")
    
    sMASTERDIR = DOINGDIR / _astro_utilities.master_dir
    REDUCEDDIR = DOINGDIR / _astro_utilities.reduced_dir
    REDUC_nightsky = DOINGDIR / _astro_utilities.reduced_nightsky_dir

    if not sMASTERDIR.exists():
        os.makedirs(str(sMASTERDIR))
        print("{} is created...".format(str(sMASTERDIR)))

    if not REDUCEDDIR.exists():
        os.makedirs(str(REDUCEDDIR))
        print("{} is created...".format(str(REDUCEDDIR)))

    if not REDUC_nightsky.exists():
        os.makedirs("{}".format(str(REDUC_nightsky)))
        print("{} is created...".format(str(REDUC_nightsky)))

    summary = yfu.make_summary(DOINGDIR/"*.fit*")
    if summary is not None :
        #print(summary)
        print("len(summary):", len(summary))
        print("summary:", summary)
        #print(summary["file"][0])

        df_light = summary.loc[summary["IMAGETYP"] == "LIGHT"].copy()
        df_light = df_light.reset_index(drop=True)

        for _, row in df_light.iterrows():

            fpath = Path(row["file"])
            ccd = yfu.load_ccd(fpath)
            filt = ccd.header["FILTER"]
            expt = ccd.header["EXPTIME"]
            if (REDUCEDDIR/ fpath.name).exists() :
                print(f"reduction file already exists...\n{fpath.name}")
                pass
            else :
                try : 
                    red = yfu.ccdred(
                        ccd,
                        output=Path(f"{REDUCEDDIR/ fpath.name}"),
                        mdarkpath=str(MASTERDIR / "master_dark_{:.0f}sec.fits".format(expt)),
                        mflatpath=str(MASTERDIR / "master_flat_{}_norm.fits".format(filt.upper())),
                        # flat_norm_value=1,  # 1 = skip normalization, None = normalize by mean
                        overwrite=True
                        )
                except FileNotFoundError: 
                    # _Python_utilities.write_log(err_log_file, "FileNotFoundError")
                    pass

Starting: CM-CYG_LIGHT_-_2016-10-14_-_RiLA600_STX-16803_-_2bin
All 97 keywords (guessed from /mnt/Rdata/OBS_data/2016-Variable/-_-_-_2016-_-_RiLA600_STX-16803_-_2bin/CM-CYG_LIGHT_-_2016-10-14_-_RiLA600_STX-16803_-_2bin/CM-CYG_LIGHT_V_2016-10-14-12-42-17_30sec_RiLA600_STX-16803_-19c_2bin.fit) will be loaded.


DELMAG   =   0.0000            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.5755            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.6398            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.6310            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.6328            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.6036            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.6340            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.6307            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.6727            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.6428            /Veriable - Comparison,   Delmag 

len(summary): 60
summary:                                                  file  filesize  SIMPLE  \
0   /mnt/Rdata/OBS_data/2016-Variable/-_-_-_2016-_...   8432640    True   
1   /mnt/Rdata/OBS_data/2016-Variable/-_-_-_2016-_...   8432640    True   
2   /mnt/Rdata/OBS_data/2016-Variable/-_-_-_2016-_...   8432640    True   
3   /mnt/Rdata/OBS_data/2016-Variable/-_-_-_2016-_...   8432640    True   
4   /mnt/Rdata/OBS_data/2016-Variable/-_-_-_2016-_...   8432640    True   
5   /mnt/Rdata/OBS_data/2016-Variable/-_-_-_2016-_...   8432640    True   
6   /mnt/Rdata/OBS_data/2016-Variable/-_-_-_2016-_...   8432640    True   
7   /mnt/Rdata/OBS_data/2016-Variable/-_-_-_2016-_...   8432640    True   
8   /mnt/Rdata/OBS_data/2016-Variable/-_-_-_2016-_...   8432640    True   
9   /mnt/Rdata/OBS_data/2016-Variable/-_-_-_2016-_...   8432640    True   
10  /mnt/Rdata/OBS_data/2016-Variable/-_-_-_2016-_...   8432640    True   
11  /mnt/Rdata/OBS_data/2016-Variable/-_-_-_2016-_...   8432640    True   

reduction file already exists...
CM-CYG_LIGHT_V_2016-10-14-12-43-43_30sec_RiLA600_STX-16803_-20c_2bin.fit
reduction file already exists...
CM-CYG_LIGHT_V_2016-10-14-12-44-25_40sec_RiLA600_STX-16803_-20c_2bin.fit


reduction file already exists...
CM-CYG_LIGHT_V_2016-10-14-12-45-18_40sec_RiLA600_STX-16803_-19c_2bin.fit
reduction file already exists...
CM-CYG_LIGHT_V_2016-10-14-12-46-11_40sec_RiLA600_STX-16803_-19c_2bin.fit


...

reduction file already exists...
CM-CYG_LIGHT_V_2016-10-14-13-59-49_80sec_RiLA600_STX-16803_-19c_2bin.fit
reduction file already exists...
CM-CYG_LIGHT_V_2016-10-14-14-01-22_80sec_RiLA600_STX-16803_-19c_2bin.fit


DELMAG   =  -0.6375            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]


reduction file already exists...
CM-CYG_LIGHT_V_2016-10-14-14-02-58_80sec_RiLA600_STX-16803_-19c_2bin.fit


DELMAG   =  -0.6055            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.5942            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]


reduction file already exists...
CM-CYG_LIGHT_V_2016-10-14-14-04-31_80sec_RiLA600_STX-16803_-19c_2bin.fit
reduction file already exists...
CM-CYG_LIGHT_V_2016-10-14-14-06-08_80sec_RiLA600_STX-16803_-20c_2bin.fit


In [21]:
if 'FILTER' in df_light :
    check_filters = df_light['FILTER'].drop_duplicates()
    check_filters = check_filters.reset_index(drop=True)
print("check_filters", check_filters)

check_filters 0    B
1    R
2    V
Name: FILTER, dtype: object


In [ ]:
for filt in check_filters:
#for filt in ["V"]:
    df_light_filt = df_light.loc[df_light["FILTER"] == filt].copy()
    
    if df_light_filt.empty:
        print(f"The dataframe(df_light_filt) {filt} is empty")
        pass
    else:

        print("len(df_light_filt):", len(df_light_filt))
        print("df_light_filt:", df_light_filt)
    
        if (sMASTERDIR / f"nightskyflat-{filt}.fits").exists() :
            pass
        else :      
            File_Num = 80
            if len(df_light_filt["file"]) > File_Num :
                combine_lst = df_light_filt["file"].tolist()[:File_Num]
            else : 
                combine_lst = df_light_filt["file"].tolist()
            try : 
                ccd = yfu.imcombine(
                                    combine_lst, 
                                    combine="med",
                                    scale="avg", 
                                    scale_to_0th=False, 
                                    reject="sc", 
                                    sigma=2.5,
                                    verbose=True,
                                    memlimit = 2.e+11,
                                    )
                ccd.write(sMASTERDIR / f"nightskyflat-{filt}.fits", overwrite=True)
                print (f"Create Create nightskyflat-{filt}.fits +++...")
            except :
                ccd = yfu.imcombine(
                                    combine_lst, 
                                    combine="med",
                                    scale="avg", 
                                    scale_to_0th=False, 
                                    reject="sc", 
                                    # sigma=2.5,
                                    verbose=True,
                                    memlimit = 2.e+11,
                                    )
                ccd.write(sMASTERDIR / f"nightskyflat-{filt}.fits", overwrite=True)
                print (f"Create Create nightskyflat-{filt}.fits +++...")

for _, row in df_light.iterrows():
    fpath = Path(row["file"])
    filt = row["FILTER"]
    if (REDUC_nightsky / fpath.name).exists() :
        print(f"Nightsky reduction file already exists...\n{fpath.name}")
        pass
    else :
        try:    
            ccd = yfu.ccdred(
                            fpath, 
                            mflatpath=str(sMASTERDIR / f"nightskyflat-{filt}.fits"),
                            output=REDUC_nightsky/fpath.name
                        )
        except FileNotFoundError: 
            # _Python_utilities.write_log(err_log_file, "FileNotFoundError") 
            pass

The dataframe(df_light_filt) B is empty
len(df_light_filt): 60
df_light_filt:                                                  file  filesize  SIMPLE  \
0   /mnt/Rdata/OBS_data/2016-Variable/-_-_-_2016-_...   8432640    True   
1   /mnt/Rdata/OBS_data/2016-Variable/-_-_-_2016-_...   8432640    True   
2   /mnt/Rdata/OBS_data/2016-Variable/-_-_-_2016-_...   8432640    True   
3   /mnt/Rdata/OBS_data/2016-Variable/-_-_-_2016-_...   8432640    True   
4   /mnt/Rdata/OBS_data/2016-Variable/-_-_-_2016-_...   8432640    True   
5   /mnt/Rdata/OBS_data/2016-Variable/-_-_-_2016-_...   8432640    True   
6   /mnt/Rdata/OBS_data/2016-Variable/-_-_-_2016-_...   8432640    True   
7   /mnt/Rdata/OBS_data/2016-Variable/-_-_-_2016-_...   8432640    True   
8   /mnt/Rdata/OBS_data/2016-Variable/-_-_-_2016-_...   8432640    True   
9   /mnt/Rdata/OBS_data/2016-Variable/-_-_-_2016-_...   8432640    True   
10  /mnt/Rdata/OBS_data/2016-Variable/-_-_-_2016-_...   8432640    True   
11  /mnt/Rdata/OBS_dat

DELMAG   =   0.0000            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.5755            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.6398            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.6310            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.6328            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.6036            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.6340            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.6307            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.6727            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.6428            /Veriable - Comparison,   Delmag 

Done.
- Loading, calculating offsets with zero/scale... 

Done.
HISTORY  Loaded 60 FITS, calculated zero, scale, weights
HISTORY  .................................(dt = 11.023 s) 2024-09-14T04:36:46.605
- Rejection
-- reject='sigclip' (irafmode=True)
--       params: nkeep=1, maxrej=60, maxiters=50, cenfunc='median'
  (for sigclip): sigma=2.5, ddof=1
Done.
- Combining
-- combine = med
Done.
HISTORY  Rejection and combination done
HISTORY  ................................(dt = 169.355 s) 2024-09-14T04:39:35.979

- Writing output FITS... Done.

2024-09-14 04:39:36.067 (TOTAL dt = 1.83e+02 sec)
Create Create nightskyflat-V.fits +++...
The dataframe(df_light_filt) R is empty
The dataframe(df_light_filt) L is empty
The dataframe(df_light_filt) R is empty
The dataframe(df_light_filt) G is empty
The dataframe(df_light_filt) B is empty


DELMAG   =   0.0000            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]


...

HISTORY  [yfu.flatcor] Flat pixels with `value < flat_mask = 0` are replaced by `flat_fill = 1`.
HISTORY  .................................................2024-09-14T04:39:57.605
HISTORY  [yfu.flatcor] Flat corrected (FLATFRM = /mnt/Rdata/OBS_data/2016-Variable/-_-_-_2016-_-_RiLA600_STX-16803_-_2bin/CM-CYG_LIGHT_-_2016-10-14_-_RiLA600_STX-16803_-_2bin/master_files_ys/nightskyflat-V.fits)
HISTORY  ..................................(dt = 0.022 s) 2024-09-14T04:39:57.620
Writing FITS to /mnt/Rdata/OBS_data/2016-Variable/-_-_-_2016-_-_RiLA600_STX-16803_-_2bin/CM-CYG_LIGHT_-_2016-10-14_-_RiLA600_STX-16803_-_2bin/reduced_nightsky/CM-CYG_LIGHT_V_2016-10-14-14-02-58_80sec_RiLA600_STX-16803_-19c_2bin.fit... Saved.


DELMAG   =  -0.6055            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =   0.0000            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]


HISTORY  [yfu.flatcor] Flat pixels with `value < flat_mask = 0` are replaced by `flat_fill = 1`.
HISTORY  .................................................2024-09-14T04:39:58.180
HISTORY  [yfu.flatcor] Flat corrected (FLATFRM = /mnt/Rdata/OBS_data/2016-Variable/-_-_-_2016-_-_RiLA600_STX-16803_-_2bin/CM-CYG_LIGHT_-_2016-10-14_-_RiLA600_STX-16803_-_2bin/master_files_ys/nightskyflat-V.fits)
HISTORY  ..................................(dt = 0.018 s) 2024-09-14T04:39:58.192
Writing FITS to /mnt/Rdata/OBS_data/2016-Variable/-_-_-_2016-_-_RiLA600_STX-16803_-_2bin/CM-CYG_LIGHT_-_2016-10-14_-_RiLA600_STX-16803_-_2bin/reduced_nightsky/CM-CYG_LIGHT_V_2016-10-14-14-04-31_80sec_RiLA600_STX-16803_-19c_2bin.fit... Saved.


DELMAG   =  -0.5942            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =   0.0000            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]


HISTORY  [yfu.flatcor] Flat pixels with `value < flat_mask = 0` are replaced by `flat_fill = 1`.
HISTORY  .................................................2024-09-14T04:39:58.518
HISTORY  [yfu.flatcor] Flat corrected (FLATFRM = /mnt/Rdata/OBS_data/2016-Variable/-_-_-_2016-_-_RiLA600_STX-16803_-_2bin/CM-CYG_LIGHT_-_2016-10-14_-_RiLA600_STX-16803_-_2bin/master_files_ys/nightskyflat-V.fits)
HISTORY  ..................................(dt = 0.019 s) 2024-09-14T04:39:58.531
Writing FITS to /mnt/Rdata/OBS_data/2016-Variable/-_-_-_2016-_-_RiLA600_STX-16803_-_2bin/CM-CYG_LIGHT_-_2016-10-14_-_RiLA600_STX-16803_-_2bin/reduced_nightsky/CM-CYG_LIGHT_V_2016-10-14-14-06-08_80sec_RiLA600_STX-16803_-20c_2bin.fit... Saved.


## Nightskt

In [19]:
trynightsky = True 

In [20]:
if trynightsky == True : 
    REDUC_nightsky = DOINGDIR / _astro_utilities.reduced_nightsky_dir
    if not REDUC_nightsky.exists():
        os.makedirs("{}".format(str(REDUC_nightsky)))
        print("{} is created...".format(str(REDUC_nightsky)))

    summary = yfu.make_summary(REDUCEDDIR /"*.fit*")
    if summary is not None :
        print("len(summary):", len(summary))
        print("summary:", summary)
        #print(summary["file"][0])   

        df_light = summary.loc[summary["IMAGETYP"] == "LIGHT"].copy()
        df_light = df_light.reset_index(drop=True)

DELMAG   =   0.0000            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.1773            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.1971            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.1899            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]


All 101 keywords (guessed from /mnt/Rdata/OBS_data/C1-Variable/-_-_-_2017-05_-_RiLA600_STX-16803_-_2bin/NSV-7244_LIGHT_-_2017-05-19_-_RiLA600_STX-16803_-_2bin/reduced/NSV-7244_LIGHT_B_2017-05-19-12-05-03_70sec_RiLA600_STX-16803_-20c_2bin.fit) will be loaded.


DELMAG   =  -0.1942            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.1855            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.1688            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.1805            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.1836            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.1787            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.1823            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.1763            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.1861            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.1791            /Veriable - Comparison,   Delmag 

len(summary): 675
summary:                                                   file  filesize  SIMPLE  \
0    /mnt/Rdata/OBS_data/C1-Variable/-_-_-_2017-05_...  16824960    True   
1    /mnt/Rdata/OBS_data/C1-Variable/-_-_-_2017-05_...  16824960    True   
2    /mnt/Rdata/OBS_data/C1-Variable/-_-_-_2017-05_...  16824960    True   
3    /mnt/Rdata/OBS_data/C1-Variable/-_-_-_2017-05_...  16824960    True   
4    /mnt/Rdata/OBS_data/C1-Variable/-_-_-_2017-05_...  16824960    True   
..                                                 ...       ...     ...   
670  /mnt/Rdata/OBS_data/C1-Variable/-_-_-_2017-05_...  16824960    True   
671  /mnt/Rdata/OBS_data/C1-Variable/-_-_-_2017-05_...  16824960    True   
672  /mnt/Rdata/OBS_data/C1-Variable/-_-_-_2017-05_...  16824960    True   
673  /mnt/Rdata/OBS_data/C1-Variable/-_-_-_2017-05_...  16824960    True   
674  /mnt/Rdata/OBS_data/C1-Variable/-_-_-_2017-05_...  16824960    True   

     BITPIX  NAXIS  NAXIS1  NAXIS2             FITS-TLM OBSE

DELMAG   =  -0.4750            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.4887            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.5066            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]
DELMAG   =  -0.5426            /Veriable - Comparison,   Delmag                  [astropy.io.fits.card]


In [ ]:
if trynightsky == True : 
    REDUC_nightsky = DOINGDIR / _astro_utilities.reduced_nightsky_dir
    if not REDUC_nightsky.exists():
        os.makedirs("{}".format(str(REDUC_nightsky)))
        print("{} is created...".format(str(REDUC_nightsky)))

    summary = yfu.make_summary(REDUCEDDIR /"*.fit*")
    if summary is not None :
        print("len(summary):", len(summary))
        print("summary:", summary)
        #print(summary["file"][0])   

        df_light = summary.loc[summary["IMAGETYP"] == "LIGHT"].copy()
        df_light = df_light.reset_index(drop=True)

        for filt in ["B", "V", "R", "L", "R", "G", "B", "b", "v", "r"]:
        #for filt in ["V"]:
            df_light_filt = df_light.loc[df_light["FILTER"] == filt].copy()
            
            if df_light_filt.empty:
                print(f"The dataframe(df_light_filt) {filt} is empty")
                pass
            else:

                print("len(df_light_filt):", len(df_light_filt))
                print("df_light_filt:", df_light_filt)
            
                if (sMASTERDIR / f"nightskyflat-{filt}.fits").exists() and not tryagain :
                    pass
                else :      
                    File_Num = 80
                    if len(df_light_filt["file"]) > File_Num :
                        combine_lst = df_light_filt["file"].tolist()[:File_Num]
                    else : 
                        combine_lst = df_light_filt["file"].tolist()
                    if (sMASTERDIR / f"nightskyflat-{filt}_norm.fits").exists():
                        pass
                    else :
                        try : 
                            ccd = yfu.imcombine(
                                                combine_lst, 
                                                combine="med",
                                                scale="avg", 
                                                scale_to_0th=False, #norm
                                                reject="sc", 
                                                sigma=2.5,
                                                verbose=True,
                                                memlimit = 2.e+11,
                                                )
                        except :
                            ccd = yfu.imcombine(
                                                combine_lst, 
                                                combine="med",
                                                scale="avg", 
                                                scale_to_0th=False, #norm
                                                reject="sc", 
                                                # sigma=2.5,
                                                verbose=True,
                                                memlimit = 2.e+11,
                                                )
                        ccd.write(sMASTERDIR / f"nightskyflat-{filt}_norm.fits", overwrite=True)
                        print (f"Create Create nightskyflat-{filt}_norm.fits +++...")

        for _, row in df_light.iterrows():
            fpath = Path(row["file"])
            ccd = yfu.load_ccd(REDUCEDDIR / fpath.name)
            filt = row["FILTER"]
            if (REDUC_nightsky / fpath.name).exists() and not tryagain:
                print(f"Nightsky reduction file already exists...\n{fpath.name}")
                pass
            else :
                try:    
                    ccd = yfu.ccdred(
                                    ccd, 
                                    mflatpath = sMASTERDIR / f"nightskyflat-{filt}_norm.fits",
                                    output = REDUC_nightsky / fpath.name
                                )
                except : 
                    # _Python_utilities.write_log(err_log_file, "FileNotFoundError") 
                    pass